In [1]:
!pip install sentence-transformers datasets
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 857 kB/s 
     |████████████████████████████████| 451 kB 51.4 MB/s 
     |████████████████████████████████| 5.8 MB 58.0 MB/s 
     |████████████████████████████████| 1.3 MB 25.7 MB/s 
     |████████████████████████████████| 182 kB 49.4 MB/s 
     |████████████████████████████████| 7.6 MB 56.3 MB/s 
     |████████████████████████████████| 212 kB 53.1 MB/s 
     |████████████████████████████████| 132 kB 60.8 MB/s 
     |████████████████████████████████| 127 kB 49.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3015cc9ef206264819cdbdc682fb7fc19cb5ea4f0b66f884bfa287710e0f22b9
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
  Attempting uninstall: urllib3
    Found existing i

In [33]:

!pip install --upgrade pip
!pip install SpeechRecognition
!pip install pyautogui
!pip install clipboard
!pip install keyboard
!pip install pyaudio

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
D:\anaconda\envs\mldl\python.exe -m pip install --upgrade pip


  Using cached SpeechRecognition-3.9.0-py2.py3-none-any.whl (32.8 MB)
  Using cached PyAutoGUI-0.9.53-py3-none-any.whl
  Using cached PyScreeze-0.1.28-py3-none-any.whl
  Using cached pytweening-1.0.4-py3-none-any.whl
  Using cached MouseInfo-0.1.3-py3-none-any.whl
  Using cached PyGetWindow-0.0.9-py3-none-any.whl
  Using cached PyMsgBox-1.0.9-py3-none-any.whl
  Using cached PyRect-0.2.0-py2.py3-none-any.whl
  Using cached pyperclip-1.8.2-py3-none-any.whl
  Using cached clipboard-0.0.4-py3-none-any.whl
  Using cached keyboard-0.13.5-py3-none-any.whl (58 kB)
  Using cached PyAudio-0.2.12-cp38-cp38-win_amd64.whl (163 kB)


In [36]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sklearn.cluster import KMeans

from speech_recognition import *
from pyautogui import *
import clipboard
import keyboard
import pyaudio
import time # 필요한 모듈 불러오기
from datetime import datetime

# AttributeError: module 'datetime' has no attribute 'now' 이 오류 나오면
# from datetime import datetime


In [37]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [38]:
# 모델 설정
# 활용할 모델을 선택
# 본 노트북에서는 klue-roberta-base 모델을 활용하지만, 
# https://huggingface.co/klue
# 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

'''
klue/roberta-base
klue/roberta-large
klue/roberta-small
klue/bert-base
'''

model_name = "klue/roberta-base"

train_batch_size = 32
num_epochs = 1
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

embedding_model = models.Transformer(model_name)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

In [39]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooler])
datasets = load_dataset("klue", "sts")
testsets = load_dataset("kor_nlu", "sts")

train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

2022-12-13 01:00:28 - Use pytorch device: cuda
2022-12-13 01:00:32 - Found cached dataset klue (C:/Users/maili/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

2022-12-13 01:00:36 - Found cached dataset kor_nlu (C:/Users/maili/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

# model.fit(
#     train_objectives=[(train_dataloader, train_loss)],
#     evaluator=evaluator,
#     epochs=num_epochs,
#     evaluation_steps=1000,
#     warmup_steps=warmup_steps,
#     output_path=model_save_path,
# )

2022-12-13 01:00:37 - Warmup-steps: 37


In [46]:
torch.save(model, "./save.pt")

In [47]:
model = torch.load("./save.pt")



In [48]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

2022-12-13 01:03:00 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-base-2022-12-13_01-00-25


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-639750b4-268e8ce54bf64aea1c71cc4b)

Repository Not Found for url: https://huggingface.co/api/models/output/training_klue_sts_klue-roberta-base-2022-12-13_01-00-25.
Please make sure you specified the correct `repo_id` and `repo_type`.
If the repo is private, make sure you are authenticated.
Invalid username or password.

In [14]:
import os
docs = []
dic = {}

dir_path = "./data/"

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        
        print((file_path.split('\\')[-1]).split('.txt')[0],"\n")
        dic[file_path] = 0
    for i in dic:
        file= open(i, 'rt' , encoding='UTF8')
        passwords = file.readlines()
        for password in passwords:
            password = password.replace('\n', '')
            docs.append(password)        
        docs=[]
        
result = [] 
for key, value in dic.items():
    if value ==0:
        result.append((key.split('\\')[-1]).split('.txt')[0])
        
print(result)        
#document_embeddings = model.encode(docs)
info =""
for i in range(len(result)):
    if result[i] == "압좌증후군":
        file= open('./data2/압좌증후군.txt', 'rt' , encoding='UTF8')
        passwords = file.readlines()
        for password in passwords:
            #print(password)
            info += password
print(info)
info = ""

심근경색 

심정지 

압좌증후군 

골절 

['심근경색', '심정지', '압좌증후군', '골절']
압좌증후군
무거운 물체에 장시간 깔리면 산소 공급이 중단돼 근육조직과 세포 등이 괴사한다. 
이때 갑자기 무거운 물체를 제거하면
괴사한 세포에서 흘러나온 칼륨이나 이오글로빈 단백질 , 젖산 등 독성 물질이 
급속히 혈액 속으로 흘러 들어가게 되고 
혈관의 자동조절 장애를 일으켜 칼륨의 상승과 산혈증으로 부정맥 발생, 
환자는 구출 후 얼마 되지 않아 사망할 수 있다.
출혈이 함께 동반되고 있다면 골든 타임이 더 짧아진다.


조치
신고자에게 환자에게 무거운 물체에 깔린 시간을 물어볼 것
40분 이상이거나 신고자가 정확히 얼마나 깔려 있었는지 기억하지 못한다면
절대 무거운 물체를 함부로 들어 올리지 말 것
무거운 물체를 들어올렸다면 
괴사한 부위에 혈액 순환 속도를 늦추기 위해 구급 대원 도착까지
깔려 있던 부위를 묶을 수 있는 수건, 윗옷 등을 이용하여
묶으라고 지시할 것 


진단
혈액 검사, 소변 검사 등으로 근육 손상을 파악 
X-ray 를 통한 골절된 부위 파악
CT, MRI 를 통한 복부 장기 출혈 파악


치료
구급대원은 환자에게 의사의 진찰까지 수액을 투여 할 것
이요글로블린에 의한 신부전이 동반을 막기 위하여 환자에게 만니톨을 투여할 것
복막염 등의 감염을 치료하기 위해 항생제를 투여할 것
무거운 물체가 깔린 정도를 파악하고 늑골 골절 예상시 폐가 손상되었을 수 있으니 인공호흡기를 환자에게 바로 가져갈 것


병원
혈액 검사, 소변 검사 및 X-ray , CT,  MRI ,혈액 투석기가 구비되어 있는 병원



In [17]:
def State(voice):
    query = voice
    query_embedding = model.encode(query)
    top_k = min(len(docs), len(docs))
    # 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
    cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

    # 코사인 유사도 순으로 `top_k` 개 문장 추출
    top_results = torch.topk(cos_scores, k=top_k)
    
    print(f"입력 문장: {query}")
    print(f"\n<입력 문장과 비교한 {top_k} 개의 문장>\n")
    
    for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
        if score > 0.5:
            print(f"{i+1}: {docs[idx]} \n{'(유사도: {:.4f})'.format(score)}\n")
    
def read_voice(): # 음성 인식을 하는 함수
    r = Recognizer()
    mic = Microphone() # 마이크 객체
    with mic as source:
        audio = r.listen(source) # 음성 읽어오기
    voice_data = r.recognize_google(audio, language='ko')
    return voice_data # 값 반환



voice = read_voice() # 음성 인식
State(voice)
# 버스안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 가래가 나와요 



KeyboardInterrupt: 

In [57]:
query = "119입니다 무엇을 도와 드릴까요 아이가 집에 왔더니 식탁에 다리가 깔려서 있습니다. 의식이 없고 혼자 힘으로 들어올릴 수 가 없습니다. 깔린지 얼마나 되었나요? 잘 모르겠습니다"
query_embedding = model.encode(query)


top_k = min(len(docs), len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 비교한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    if score > 0.4:
        print(f"{i+1}: {docs[idx]} \n{'(유사도: {:.4f})'.format(score)}\n")



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

입력 문장: 119입니다 무엇을 도와 드릴까요 아이가 집에 왔더니 식탁에 다리가 깔려서 있습니다. 의식이 없고 혼자 힘으로 들어올릴 수 가 없습니다. 깔린지 얼마나 되었나요? 잘 모르겠습니다

<입력 문장과 비교한 9 개의 문장>

1: 119입니다. 지금 공사장 기구에 사람이 깔렸습니다. 의식이 없어요. 깔린 시간이 얼마나 되나요? 잘 모르겠습니다. 함부로 기구를 들어서 환자를 꺼내시면 안됩니다 
(유사도: 0.6923)

2: 집에 왔는데 아이가 서랍에 깔렸어요. 의식이 없습니다 
(유사도: 0.5041)

3: 지진에 사람이 깔렸어요. 의식이 없습니다 
(유사도: 0.4799)

4: 지진 때문에 건물이 무너져서 사람이 깔렸습니다. 의식을 잃었어요 
(유사도: 0.4797)

5: 동생이 길을 가다가 교통사고가 났습니다 다리가 골절 된거 같아요 
(유사도: 0.4160)



In [17]:
# 되긴 하는데 외국어만 되는 상황 
# m4a 를 wav 로 바꿀 방법을 찾아야 한다.
# 더럽게 없다...
import speech_recognition as sr
import os


# 파일 확장자와 이름 바꾸기
file_path = 'D:\\nlp\\audio\\'
file_names = os.listdir(file_path)
file_names    
print(file_names)

i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.wav'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1


r = sr.Recognizer()

harvard = sr.AudioFile('D:\\nlp\\audio\\'+file_names[0])
with harvard as source:
    audio = r.record(source)
    
vToText = r.recognize_google(audio_data=audio , language="ko-KR")
# (audio_data=choi_jtbc, language="ko-KR")   
print(vToText)


['1.wav']


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [8]:
#읽을떄
f=open("C:\\Users\\maili\\OneDrive\\문서\\소리 녹음\\1.m4a","rb")
c=f.read()
f.close() 
#썼으면 잘 닫아주고
print(c)

#쓰기
f=open("C:\\Users\\maili\\OneDrive\\문서\\소리 녹음\\1111.wav","wb")
f.write(c)
f.close() 

b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\x0e\xca\n\x95\x96\x00\x00\x00\x0c10.0.19045.0\x00\x01h\x15mdat\x00\x00\x00\x00\x00\x00\x00\x10\'\x0cC \x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x044\x97\xe4\x00\x00\x00\x00\x00\x00\x1fO\xa7\xd3\xe9\xf4\xfa}>\x9fO\xa7\xd3\xe9\xf4\xfa}>\x9fO\xa7\xd3\xe9\xf4\xfa}5\xd1E\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14~\xbf_\xaf\xd7\xeb\xf5\xfa\xfd~\xbf_\xaf\xd7\xeb\xf5\xfa\xfd~\xbf_\xaf\xd7\xeb\xf5\xfa\xfdp\'\x0cC \x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x

In [ ]:
### 아레 코드 절대 실행시키지 말것 실행시켰다면 output 파일 두개랑 

In [20]:
document_embeddings = model.encode(docs)

num_clusters = 3

k_means = KMeans(n_clusters=num_clusters)
k_means.fit(document_embeddings)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

D:\anaconda\envs\mldl\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=3)

In [21]:
cluster_assignment = k_means.labels_

In [22]:
cluster_assignment

array([0, 0, 2, 0, 0, 2, 2, 1])

In [43]:
import torch
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Huffon/sentence-klue-roberta-base")

docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]
document_embeddings = model.encode(docs)

query = "손흥민은 어린 나이에 유럽에 진출하였다."
query_embedding = model.encode(query)

top_k = min(5, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

2022-12-09 22:10:04 - Load pretrained SentenceTransformer: Huffon/sentence-klue-roberta-base
2022-12-09 22:10:14 - No sentence-transformers model found with name C:\Users\maili/.cache\torch\sentence_transformers\Huffon_sentence-klue-roberta-base. Creating a new one with MEAN pooling.
2022-12-09 22:10:15 - Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

입력 문장: 손흥민은 어린 나이에 유럽에 진출하였다.

<입력 문장과 유사한 5 개의 문장>

1: 독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다. (유사도: 0.5898)

2: 그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다. (유사도: 0.4858)

3: 1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다. (유사도: 0.4047)

4: 함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다. (유사도: 0.3953)

5: 춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다. (유사도: 0.3182)

